In [2]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from sklearn.model_selection import train_test_split

train_dir = "..\isec-ic\\dataset\\train"
validation_dir = "..\isec-ic\\dataset\\valid"
test_dir = "..\isec-ic\\dataset\\test"

current_dir = os.path.dirname(os.path.realpath(__file__ if '__file__' in locals() else os.getcwd()))
train_dir = os.path.join(current_dir, train_dir)
validation_dir = os.path.join(current_dir, validation_dir)
test_dir = os.path.join(current_dir, test_dir)

categories = ["bacterialspot", "healthy","lateblight","leafmold", "mosaicvirus","yellowleafcurlvirus" ,"spidermite","septorialeafspot" ]

def load_images_from_folder(folder):
    images = []
    labels = []
    subfolders = os.listdir(folder)
    for subfolder in subfolders:
        category = subfolder.split("_")[-1].lower()
        if category in categories:
            print("Loading images from category: " + category + " and subfolder: " + subfolder)
            subfolder_path = os.path.join(folder, subfolder)
            for filename in os.listdir(subfolder_path):
                img = Image.open(os.path.join(subfolder_path, filename))
                img = img.resize((64, 64))
                img = np.array(img) / 255.0
                images.append(img)
                labels.append(categories.index(category))
        else:
            print("Category: " + category + " from subfolder " + subfolder + " is not in the list of categories.")
    return np.array(images), np.array(labels)


x_train, y_train = load_images_from_folder(train_dir)
x_validation, y_validation = load_images_from_folder(validation_dir)
x_test, y_test = load_images_from_folder(test_dir)

print("Images loaded successfully")

reduction_ratio = 0.8

x_validation, x_reduction_validation, y_validation, y_reduction_validation = train_test_split(x_validation, y_validation, test_size=reduction_ratio, random_state=42)
print("Conjunto de validação reduzido em 80%")

x_train, x_reduction, y_train, y_reduction = train_test_split(x_train, y_train, test_size=reduction_ratio, random_state=42)
print("Conjunto de treino reduzido em 80%")

Loading images from category: spidermite and subfolder: Tomato__spidermite
Loading images from category: bacterialspot and subfolder: Tomato___Bacterialspot
Category: earlyblight from subfolder Tomato___Earlyblight is not in the list of categories.
Loading images from category: healthy and subfolder: Tomato___healthy
Loading images from category: lateblight and subfolder: Tomato___Lateblight
Loading images from category: leafmold and subfolder: Tomato___Leafmold
Loading images from category: septorialeafspot and subfolder: Tomato___Septorialeafspot
Category: targetspot from subfolder Tomato___Targetspot is not in the list of categories.
Loading images from category: mosaicvirus and subfolder: Tomato___Tomato_mosaicvirus
Loading images from category: yellowleafcurlvirus and subfolder: Tomato___Tomato_yellowleafcurlvirus
Loading images from category: bacterialspot and subfolder: Tomato___Bacterialspot
Category: earlyblight from subfolder Tomato___Earlyblight is not in the list of categor

In [28]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import Xception

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout

def create_cnn_model(dropout_rate, learning_rate):
    #base_model = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
    #base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(64, 64, 3)))
    base_model = Xception(weights='imagenet', include_top=False, input_tensor=Input(shape=(64, 64, 3)))

    for layer in base_model.layers:
        layer.trainable = False

    # Add the dense layers (top layers)
    x = base_model.output
    x = Flatten()(x)  # Flatten the output layer to 1 dimension
    x = Dense(128, activation='relu')(x)   #  First dense layer
    x = Dropout(dropout_rate)(x)  # Dropout to reduce overfitting
    predictions = Dense(len(categories), activation='softmax')(x)  # Output layer

    # Final model
    model = Model(inputs=base_model.input, outputs=predictions)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

print("Create_cnn_model function loaded")

Create_cnn_model function loaded


In [4]:
import os
import psutil

# Define a função para limitar a afinidade do processador
def limitar_afinidade(processo, num_cores):
    if num_cores < 1:
        num_cores = 1
    elif num_cores > psutil.cpu_count(logical=False):
        num_cores = psutil.cpu_count(logical=False)

    processo.cpu_affinity(list(range(num_cores)))

# Obtém o ID do processo atual
pid = os.getpid()

# Define o número de núcleos desejado (metade dos núcleos disponíveis)
metade_dos_cores = psutil.cpu_count(logical=False) // 4

# Limita a afinidade do processador
limitar_afinidade(psutil.Process(pid), metade_dos_cores)

# Restaura a afinidade do processador para todos os núcleos após a execução do código
limitar_afinidade(psutil.Process(pid), psutil.cpu_count(logical=False))


if tf.test.is_gpu_available():
    print("GPU disponível. Usando GPU para treinamento.")
else:
    print("GPU não disponível. Usando CPU para treinamento.")

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU disponível. Usando GPU para treinamento.


In [33]:
from swarm.pso import pso
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

print("Iniciando a otimização com PSO...")
dimension = 2    # Número de dimensões (neste caso, dropout_rate e learning_rate)
lb = [0.1, 0.0001]   # Limites inferiores para dropout_rate e learning_rate
ub = [1, 0.01]    # Limites superiores para dropout_rate e learning_rate
num_agents = 2  # Número de agentes

max_iter = 10   # Número de iterações
total_epochs = 10
k_n_splits = 3 # k splits 

best_auc_median_score = 0  # Variável global para armazenar o melhor auc_score

def evaluate_model_PSO(solution):
    global best_auc_median_score
    print(f" - Evaluating with cross validation, splits={k_n_splits}")
    dropout_rate, learning_rate = solution
    dropout_rate = abs(dropout_rate)
    learning_rate = abs(learning_rate)
    kfold = KFold(n_splits=k_n_splits, shuffle=True, random_state=42) 
    auc_scores = []

    for train, val in kfold.split(x_train, y_train):
        model = create_cnn_model(dropout_rate, learning_rate)
        
        # Treina o modelo com os dados de treino e valida nos dados de validação
        model.fit(x_train[train], y_train[train], epochs=total_epochs, batch_size=32, verbose=0)
        y_pred = model.predict(x_train[val])
        y_val_categorical = to_categorical(y_train[val], num_classes=len(categories))
        auc_score = roc_auc_score(y_val_categorical, y_pred, multi_class='ovr')
        auc_scores.append(auc_score)
        print(f"   -- generating model with -> dropout_rate: {dropout_rate}, learning_rate: {learning_rate}, auc score: {auc_score}")

    auc_median_score_maximized = np.mean(auc_scores) 
    auc_median_score_minimized = 1 - auc_median_score_maximized 
    
    if(auc_median_score_maximized >= best_auc_median_score):
        best_auc_median_score = auc_median_score_maximized
    print(f" - generated cnn models -> dropout_rate: {dropout_rate}, learning_rate: {learning_rate}, auc median score (minimized): {auc_median_score_minimized}, maximized median auc: {auc_median_score_maximized}")
    
    # Retorna a média do AUC score (minimizado) 1- auc
    return auc_median_score_minimized


# Criando a instância do PSO
pso_instance = pso(n=num_agents, function=evaluate_model_PSO, lb=lb, ub=ub, dimension=dimension, iteration=max_iter)
best_solution = pso_instance.get_Gbest()

best_dropout_rate, best_learning_rate = best_solution
best_dropout_rate = abs(best_dropout_rate)
best_learning_rate = abs(best_learning_rate)

print(f"Melhores parâmetros encontrados: \nBest dropout rate: {best_dropout_rate}, Best Learning Rate: {best_learning_rate}")
print(f"Best AUC score: {best_auc_median_score}")

Iniciando a otimização com PSO...
 - Evaluating with cross validation, splits=3
   -- generating model with -> dropout_rate: 0.6698029711913597, learning_rate: 0.008230172097056803, auc score: 0.9145670916513409
   -- generating model with -> dropout_rate: 0.6698029711913597, learning_rate: 0.008230172097056803, auc score: 0.9274767583035491
   -- generating model with -> dropout_rate: 0.6698029711913597, learning_rate: 0.008230172097056803, auc score: 0.9317614425363413
 - generated cnn models -> dropout_rate: 0.6698029711913597, learning_rate: 0.008230172097056803, auc median score (minimized): 0.07539823583625627, maximized median auc: 0.9246017641637437
 - Evaluating with cross validation, splits=3
   -- generating model with -> dropout_rate: 0.7760058412970174, learning_rate: 0.006640610984371415, auc score: 0.9024458283226837
   -- generating model with -> dropout_rate: 0.7760058412970174, learning_rate: 0.006640610984371415, auc score: 0.9143461583493295
   -- generating model w

In [ ]:
import matplotlib.pyplot as plt

#best_learning_rate = 0.003889651856290339 
#best_dropout_rate = 0.7057278672251002

x_train = x_reduction
x_validation = x_reduction_validation

y_train = y_reduction
y_validation =  y_reduction_validation

print("Treinando o modelo com os melhores hiperparâmetros...")
best_model = create_cnn_model(best_dropout_rate, best_learning_rate)
best_model_history = best_model.fit(x_train, y_train, epochs=20, batch_size=32, verbose=1, validation_data=(x_validation, y_validation))

best_pso_train_accuracy = best_model_history.history['accuracy'][-1]
best_pso_validation_accuracy = best_model_history.history['val_accuracy'][-1]

print("Melhor Accuracy de Treinamento:", best_pso_train_accuracy)
print("Melhor Accuracy de Validação:", best_pso_validation_accuracy)


# Plotar as curvas de treino e validação
plt.plot(best_model_history.history['accuracy'], label='Treinamento')
plt.plot(best_model_history.history['val_accuracy'], label='Validação')
plt.xlabel('Épocas')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
print("Avaliando o modelo no conjunto de teste...")
loss_test, test_accuracy = best_model.evaluate(x_test, y_test, verbose=0)

print(f"Acurácia no Teste: {test_accuracy}, Loss: {loss_test}")